In [1]:
using Gridap, Gridap.Geometry, Gridap.Fields, GridapGmsh
using LinearAlgebra
using NLopt
using CairoMakie, GridapMakie
using ChainRulesCore, Zygote
import ChainRulesCore: rrule
NO_FIELDS = ZeroTangent()

λ = 1.0           # Wavelength
L = 2.5           # Width of the rectangular domain
H = 2.5           # Height of the rectangular domain
dpml = 0.5        # Thickness of PML

ε1 = 1            # Air
ε2 = 12           # Silicon
μ = 1             # Magnetic permeability
k = 2*π/λ         # Wavenumber (nm^-1)


model = GmshDiscreteModel("geometry.msh")

order = 1
reffe = ReferenceFE(lagrangian, Float64, order)
V = TestFESpace(model, reffe, dirichlet_tags = ["DirichletEdges", "DirichletNodes"], vector_type = Vector{ComplexF64})
U = V   # mathematically equivalent to TrialFESpace(V,0)

degree = 2
Ω = Triangulation(model)
dΩ = Measure(Ω, degree)

Ω_d = Triangulation(model, tags="Design")
dΩ_d = Measure(Ω_d, degree)

Γ_t = BoundaryTriangulation(model; tags = "Target")
dΓ_t = Measure(Γ_t, degree)

p_reffe = ReferenceFE(lagrangian, Float64, 0)
Q = TestFESpace(Ω_d, p_reffe, vector_type = Vector{Float64})
P = Q
np = num_free_dofs(P) # Number of cells in design region (number of design parameters)

pf_reffe = ReferenceFE(lagrangian, Float64, 1)
Qf = TestFESpace(Ω_d, pf_reffe, vector_type = Vector{Float64})
Pf = Qf
fem_params = (; V, U, Q, P, Qf, Pf, np, Ω, dΩ, dΩ_d, dΓ_t)


Info    : Reading 'geometry.msh'...
Info    : 60 entities
Info    : 9325 nodes
Info    : 18930 elements
Info    : Done reading 'geometry.msh'


(V = UnconstrainedFESpace(), U = UnconstrainedFESpace(), Q = UnconstrainedFESpace(), P = UnconstrainedFESpace(), Qf = UnconstrainedFESpace(), Pf = UnconstrainedFESpace(), np = 4650, Ω = BodyFittedTriangulation(), dΩ = Measure(), dΩ_d = Measure(), dΓ_t = Measure())

In [2]:
R = 1e-10
LHp=(L/2, H/2)       # Start of PML for x,y > 0
LHn=(L/2, H/2)       # Start of PML for x,y < 0
phys_params = (; k, ε1, ε2, μ, R, dpml, LHp, LHn)

# PML coordinate streching functions
function s_PML(x; phys_params)
    σ = -3 / 4 * log(phys_params.R) / phys_params.dpml / sqrt(phys_params.ε1)
    xf = Tuple(x)
    u = @. ifelse(xf > 0 , xf - phys_params.LHp, - xf - phys_params.LHn)
    return @. ifelse(u > 0,  1 + (1im * σ / phys_params.k) * (u / phys_params.dpml)^2, $(1.0+0im))
end

function ds_PML(x; phys_params)
    σ = -3 / 4 * log(phys_params.R) / phys_params.dpml / sqrt(phys_params.ε1)
    xf = Tuple(x)
    u = @. ifelse(xf > 0 , xf - phys_params.LHp, - xf - phys_params.LHn)
    ds = @. ifelse(u > 0, (2im * σ / phys_params.k) * (1 / phys_params.dpml)^2 * u, $(0.0+0im))
    return ds.*sign.(xf)
end

struct Λ{PT} <: Function
    phys_params::PT
end

function (Λf::Λ)(x)
    s_x,s_y = s_PML(x; Λf.phys_params)
    return VectorValue(1/s_x, 1/s_y)
end

# Define the derivative for the Λ factor
Fields.∇(Λf::Λ) = x -> TensorValue{2, 2, ComplexF64}(-(Λf(x)[1])^2 * ds_PML(x; Λf.phys_params)[1], 0, 0, -(Λf(x)[2])^2 * ds_PML(x; Λf.phys_params)[2])


r = 5/sqrt(3)               # Filter radius
β = 32.0                    # β∈[1,∞], threshold sharpness
η = 0.5                     # η∈[0,1], threshold center    

a_f(r, u, v) = r^2 * (∇(v) ⋅ ∇(u))

function Filter(p0; r, fem_params)
    ph = FEFunction(fem_params.P, p0)
    op = AffineFEOperator(fem_params.Pf, fem_params.Qf) do u, v
        ∫(a_f(r, u, v))fem_params.dΩ_d + ∫(v * u)fem_params.dΩ_d, ∫(v * ph)fem_params.dΩ_d
      end
    pfh = solve(op)
    return get_free_dof_values(pfh)
end

function Threshold(pfh; β, η)
    return ((tanh(β * η) + tanh(β * (pfh - η))) / (tanh(β * η) + tanh(β * (1.0 - η))))         
end


ξd(p, ε1, ε2)= 1 / (ε1 + (ε2 - ε1) * p) - 1 / ε1 # in the design region

a_base(u, v; phys_params) = (1 / phys_params.ε1) * ((∇ .* (Λ(phys_params) * v)) ⊙ (Λ(phys_params) .* ∇(u))) - (phys_params.k^2 * phys_params.μ * (v * u))

a_design(u, v, pth; phys_params) = ((p -> ξd(p, phys_params.ε1, phys_params.ε2)) ∘ pth) * (∇(v) ⊙ ∇(u))

function MatrixA(pth; phys_params, fem_params)
    A_mat = assemble_matrix(fem_params.U, fem_params.V) do u, v
        ∫(a_base(u, v; phys_params))fem_params.dΩ + ∫(a_design(u, v, pth; phys_params))fem_params.dΩ_d
    end
    return lu(A_mat)
end

function MatrixB(pth; fem_params)
    B_mat = assemble_matrix(fem_params.U, fem_params.V) do u, v
        ∫(pth * (∇(u) ⋅ ∇(v)))fem_params.dΩ_d
    end
    return B_mat
end

# Poynting flux
Sp(u, v) = 1im / (4 * k * ε1) * (u * ∇(v) - v * ∇(u))
function MatrixO(fem_params)
    return assemble_matrix(fem_params.U, fem_params.V) do u, v
        ∫( Sp(u, v) ⋅ (x -> (x / norm(x))) )fem_params.dΓ_t
    end
end


MatrixO (generic function with 1 method)

Recall that our objective is
\begin{equation}
   g(p,W)=\mathrm{tr} [\left(A^{-1}BW\right)^\dagger O\underbrace{\left(A^{-1}BW\right)}_{U}(W^\dagger BW)^{-1} ]= \mathrm{tr}\left[U^\dagger OU(W^\dagger BW)^{-1}\right]
   \, ,
\end{equation}
where we have defined the function $g(p,W)$  for the parameter and $W$ dependence, along with $U = A^{-1} BW$


In [3]:
# Objective trace 
function g_pf(pf_vec; β, η, O_mat, W_mat, phys_params, fem_params)
    pfh = FEFunction(fem_params.Pf, pf_vec)
    pth = (pf -> Threshold(pf; β, η)) ∘ pfh
    A_mat = MatrixA(pth; phys_params, fem_params)
    B_mat = MatrixB(pth; fem_params)
    U_mat = A_mat \ (B_mat * W_mat)
    g_temp = tr((U_mat' * O_mat * U_mat) / (W_mat' * B_mat * W_mat))
    @assert abs(imag(g_temp) / real(g_temp)) < 1e-6
    real(g_temp)
end

function pf_p0(p0; r, fem_params)
    pf_vec = Filter(p0; r, fem_params)
    pf_vec[pf_vec .< 0] .= 0
    pf_vec[pf_vec .> 1] .= 1.0
    pf_vec
end


pf_p0 (generic function with 1 method)

Derivative to design parameters from adjoint method:
$$
\frac{\partial g}{\partial p}=-\mathrm{tr}\left[U^\dagger OU(W^\dagger BW)^{-1}(W^\dagger\frac{\partial B}{\partial p}W)(W^\dagger BW)^{-1}\right] -2\Re\left\{\mathrm{tr}\left[Z^\dagger\left(\frac{\partial A}{\partial p}U-\frac{\partial B}{\partial p}W\right)\right]\right\}\, ,
$$
where
$$A^\dagger Z=OU(W^\dagger BW)^{-1}$$

In [4]:
Dptdpf(pf, β, η) = β * (1.0 - tanh(β * (pf - η))^2) / (tanh(β * η) + tanh(β * (1.0 - η)))

Dξdpf(pf, ε1, ε2, β, η)= (ε1 - ε2) / (ε1 + (ε2 - ε1) * Threshold(pf; β, η))^2 * Dptdpf(pf, β, η)

DAdpf(u, v, pfh; phys_params, β, η) = ((p -> Dξdpf(p, phys_params.ε1, phys_params.ε2, β, η)) ∘ pfh) * (∇(v) ⊙ ∇(u))

DBdpf(pfh, u, v; β, η) = ((pf -> Dptdpf(pf, β, η)) ∘ pfh) * (∇(v) ⊙ ∇(u))

# dg/dpf = dg/dg * dg/dpf
function rrule(::typeof(g_pf), pf_vec; β, η, O_mat, W_mat, phys_params, fem_params)
    function U_pullback(dgdg)
      NO_FIELDS, dgdg * Dgdpf(pf_vec; β, η, O_mat, W_mat, phys_params, fem_params)
    end
    g_pf(pf_vec; β, η, O_mat, W_mat, phys_params, fem_params), U_pullback
end

function Dgdpf(pf_vec; β, η, O_mat, W_mat, phys_params, fem_params)
    pfh = FEFunction(fem_params.Pf, pf_vec)
    pth = (pf -> Threshold(pf; β, η)) ∘ pfh
    A_mat = MatrixA(pth; phys_params, fem_params)
    B_mat = MatrixB(pth; fem_params)
    
    U_mat = A_mat \ (B_mat * W_mat)
    WBW = W_mat' * B_mat * W_mat
    dgdU = (O_mat * U_mat) / WBW
    Z_mat = A_mat' \ dgdU
    Wr_mat = W_mat / WBW
    Wl_mat = W_mat * (dgdU' * U_mat)
    
    dgdpf = zeros(num_free_dofs(fem_params.Pf))
    for k_i = 1 : size(W_mat)[2]
        uh = FEFunction(fem_params.U, U_mat[:, k_i])
        zh = FEFunction(fem_params.V, conj(Z_mat[:, k_i]))
        wh = FEFunction(fem_params.U, W_mat[:, k_i])
        wrh = FEFunction(fem_params.U, Wr_mat[:, k_i])
        wlh = FEFunction(fem_params.V, conj(Wl_mat[:, k_i]))
        l_temp(dp) = ∫(real(2 * DBdpf(pfh, wh, zh; β, η) - 2 * DAdpf(uh, zh, pfh; phys_params, β, η) - DBdpf(pfh, wrh, wlh; β, η)) * dp)fem_params.dΩ_d
        dgdpf += assemble_vector(l_temp,fem_params.Pf)
    end
    return dgdpf
end

# dg/dp = dg/dpf*dpf/dp
function rrule(::typeof(pf_p0), p0; r, fem_params)
  function pf_pullback(dgdpf)
    NO_FIELDS, Dgdp(dgdpf; r, fem_params)
  end
  pf_p0(p0; r, fem_params), pf_pullback
end

function Dgdp(dgdpf; r, fem_params)
    Af = assemble_matrix(fem_params.Pf, fem_params.Qf) do u, v
        ∫(a_f(r, u, v))fem_params.dΩ_d + ∫(v * u)fem_params.dΩ_d
    end
    wvec = Af' \ dgdpf
    wh = FEFunction(fem_params.Pf, wvec)
    l_temp(dp) = ∫(wh * dp)fem_params.dΩ_d
    return assemble_vector(l_temp, fem_params.P)
end

# Final objective function
function g_p(p0::Vector; r, β, η, O_mat, W_mat, phys_params, fem_params)
    pf_vec = pf_p0(p0; r, fem_params)
    g_pf(pf_vec; β, η, O_mat, W_mat, phys_params, fem_params)
end

function g_p(p0::Vector, grad::Vector; r, β, η, O_mat, W_mat, phys_params, fem_params)
    if length(grad) > 0
        dgdp, = Zygote.gradient(p -> g_p(p; r, β, η, O_mat, W_mat, phys_params, fem_params), p0)
        grad[:] = dgdp
    end
    g_value = g_p(p0; r, β, η, O_mat, W_mat, phys_params, fem_params)
    return g_value
end

g_p (generic function with 2 methods)

Using the following code, we can verify the derivative by comparing to the finite difference results.

In [5]:
p0 = rand(fem_params.np)
δp = rand(fem_params.np)*1e-8
grad = zeros(fem_params.np)
K = 5
O_mat = MatrixO(fem_params)
W_mat = rand(ComplexF64, num_free_dofs(fem_params.U), K)

g0 = g_p(p0, grad; r, β, η, O_mat, W_mat, phys_params, fem_params)
g1 = g_p(p0+δp, []; r, β, η, O_mat, W_mat, phys_params, fem_params)
g1-g0, grad'*δp

(-2.1555374791950438e-10, -2.1555160414422312e-10)

Since $W$ is another variable to optimize, we also need the derivative to the matrix $W$:
$$
\frac{\partial g}{\partial W}=\left[I-BW(W^\dagger BW)^{-1}W^\dagger\right](A^{-1}B)^\dagger OU(W^\dagger BW)^{-1}
$$

In [6]:
function DgdW(A, W, B, O)
    WBW = W' * B * W
    U = A \ (B * W)
    B' * (A' \ (O * (U / WBW))) - (B * W / WBW) * (U' * (O * (U / WBW)))
end

# We combine p0 and complex matrix W to a real-valued vector called pW
function g_pW(pW::Vector, grad::Vector; r, β, η, K, O_mat, phys_params, fem_params)
    N = num_free_dofs(fem_params.U)
    @assert length(pW) == (fem_params.np + 2 * N * K)
    p0 = zeros(fem_params.np)
    for i = 1 : fem_params.np
        p0[i] = pW[i]
    end
    W_mat = reinterpret(ComplexF64, reshape(pW[fem_params.np + 1 : end], (2 * N, K)))

    if length(grad) > 0
        pf_vec = pf_p0(p0; r, fem_params)
        pfh = FEFunction(fem_params.Pf, pf_vec)
        pth = (pf -> Threshold(pf; β, η)) ∘ pfh
        A_mat = MatrixA(pth; phys_params, fem_params)
        B_mat = MatrixB(pth; fem_params)
        
        dgdp, = Zygote.gradient(p -> g_p(p; r, β, η, O_mat, W_mat, phys_params, fem_params), p0)
        grad[1 : fem_params.np] = dgdp

        dgdW = reinterpret(Float64, DgdW(A_mat, W_mat, B_mat, O_mat))
        grad[fem_params.np + 1 : end] = 2 * dgdW[:]
    end
    g_value = g_p(p0; r, β, η, O_mat, W_mat, phys_params, fem_params)
    open("gvalue.txt", "a") do io
        write(io, "$g_value \n")
    end
    return g_value
end

g_pW (generic function with 1 method)

Similarly, we can use the following code to verify the gradients. 

Recall that the first `np` elements in the vector `pW` is the design parameters `p0 = pW[1:np]` and the rest are matrix $W$ components (a total of $2NK$ elements since we expanded the real and imagniary part of $W$: `W_mat = reinterpret(Float64, W_mat), pW[np + 1 : end] = W_mat[:]`.)

In [7]:
N = num_free_dofs(fem_params.U)
K = 5
pW = rand(fem_params.np+2*N*K)
δpW = rand(fem_params.np+2*N*K)*1e-8
grad = zeros(fem_params.np+2*N*K)

O_mat = MatrixO(fem_params)

g0 = g_pW(pW, grad; r, β, η, K, O_mat, phys_params, fem_params)
g1 = g_pW(pW+δpW, []; r, β, η, K, O_mat, phys_params, fem_params)
g1-g0, grad'*δpW

(9.094671897091205e-10, 9.094738541499147e-10)

In [ ]:
"""
Gmsh function that creates a rectangular domain with center circle
"""

using Gmsh
import Gmsh: gmsh

struct CirRecGeometry
    L::Float64           # Length of the normal region  
    H::Float64           # Height of the air region
    rd::Float64          # Radius of design circle
    rt::Float64          # Radius the target circle
    dpml::Float64        # Thickness of the PML
    # Characteristic length (controls the resolution, smaller the finer)
    l1::Float64          # Normal region
    l2::Float64          # Design region
    l3::Float64          # PML 
end


function MeshGenerator(geo_param::CirRecGeometry, meshfile_name::String)
    gmsh.initialize()
    gmsh.option.setNumber("General.Terminal", 1)
    gmsh.option.setNumber("Mesh.Algorithm", 6)
    gmsh.clear()
    gmsh.model.add("geometry") # name it whatever you want

    # Add points
    gmsh.model.geo.addPoint(-geo_param.L/2-geo_param.dpml, -geo_param.H/2-geo_param.dpml, 0, geo_param.l3,  1)
    gmsh.model.geo.addPoint(-geo_param.L/2     , -geo_param.H/2-geo_param.dpml, 0, geo_param.l3,  2)
    gmsh.model.geo.addPoint( geo_param.L/2     , -geo_param.H/2-geo_param.dpml, 0, geo_param.l3,  3)
    gmsh.model.geo.addPoint( geo_param.L/2+geo_param.dpml, -geo_param.H/2-geo_param.dpml, 0, geo_param.l3,  4)
    gmsh.model.geo.addPoint(-geo_param.L/2-geo_param.dpml, -geo_param.H/2     , 0, geo_param.l3,  5)
    gmsh.model.geo.addPoint(-geo_param.L/2     , -geo_param.H/2     , 0, geo_param.l1,  6)
    gmsh.model.geo.addPoint( geo_param.L/2     , -geo_param.H/2     , 0, geo_param.l1,  7)
    gmsh.model.geo.addPoint( geo_param.L/2+geo_param.dpml, -geo_param.H/2   , 0, geo_param.l3,  8)
    gmsh.model.geo.addPoint(-geo_param.L/2-geo_param.dpml,  geo_param.H/2   , 0, geo_param.l3,  9)
    gmsh.model.geo.addPoint(-geo_param.L/2     , geo_param.H/2      , 0, geo_param.l1, 10)
    gmsh.model.geo.addPoint( geo_param.L/2     , geo_param.H/2      , 0, geo_param.l1, 11)
    gmsh.model.geo.addPoint( geo_param.L/2+geo_param.dpml, geo_param.H/2      , 0, geo_param.l3, 12)
    gmsh.model.geo.addPoint(-geo_param.L/2-geo_param.dpml, geo_param.H/2+geo_param.dpml , 0, geo_param.l3, 13)
    gmsh.model.geo.addPoint(-geo_param.L/2     , geo_param.H/2+geo_param.dpml , 0, geo_param.l3, 14)
    gmsh.model.geo.addPoint( geo_param.L/2     , geo_param.H/2+geo_param.dpml , 0, geo_param.l3, 15)
    gmsh.model.geo.addPoint( geo_param.L/2+geo_param.dpml, geo_param.H/2+geo_param.dpml , 0, geo_param.l3, 16)
    gmsh.model.geo.addPoint(-geo_param.rd      , 0        , 0, geo_param.l2, 17)
    gmsh.model.geo.addPoint( 0       , 0       , 0, geo_param.l2, 18)
    gmsh.model.geo.addPoint( geo_param.rd      , 0        , 0, geo_param.l2, 19)
    gmsh.model.geo.addPoint(-geo_param.rt      , 0        , 0, geo_param.l1, 20)
    gmsh.model.geo.addPoint( 0       , 0       , 0, geo_param.l1, 21)
    gmsh.model.geo.addPoint( geo_param.rt      , 0        , 0, geo_param.l1, 22)
    # Add lines
    gmsh.model.geo.addLine(  1,  2,  1)
    gmsh.model.geo.addLine(  2,  6,  2)
    gmsh.model.geo.addLine(  6,  5,  3)
    gmsh.model.geo.addLine(  5,  1,  4)
    gmsh.model.geo.addLine(  2,  3,  5)
    gmsh.model.geo.addLine(  3,  7,  6)
    gmsh.model.geo.addLine(  7,  6,  7)
    gmsh.model.geo.addLine(  3,  4,  8)
    gmsh.model.geo.addLine(  4,  8,  9)
    gmsh.model.geo.addLine(  8,  7, 10)
    gmsh.model.geo.addLine(  6, 10, 11)
    gmsh.model.geo.addLine( 10,  9, 12)
    gmsh.model.geo.addLine(  9,  5, 13)
    gmsh.model.geo.addLine(  7, 11, 14)
    gmsh.model.geo.addLine( 11, 10, 15)
    gmsh.model.geo.addLine(  8, 12, 16)
    gmsh.model.geo.addLine( 12, 11, 17)
    gmsh.model.geo.addLine( 10, 14, 18)
    gmsh.model.geo.addLine( 14, 13, 19)
    gmsh.model.geo.addLine( 13,  9, 20)
    gmsh.model.geo.addLine( 11, 15, 21)
    gmsh.model.geo.addLine( 15, 14, 22)
    gmsh.model.geo.addLine( 12, 16, 23)
    gmsh.model.geo.addLine( 16, 15, 24)
    gmsh.model.geo.addCircleArc( 17, 18, 19, 25)
    gmsh.model.geo.addCircleArc( 19, 18, 17, 26)
    gmsh.model.geo.addCircleArc( 20, 21, 22, 27)
    gmsh.model.geo.addCircleArc( 22, 21, 20, 28)
    #gmsh.model.geo.addLine( 17, 18, 25)
    #gmsh.model.geo.addLine( 18, 19, 26)
    #gmsh.model.geo.addLine( 19, 17, 31)
    # Construct curve loops and surfaces 
    gmsh.model.geo.addCurveLoop([  1,  2,  3,  4], 1)
    gmsh.model.geo.addCurveLoop([  5,  6,  7, -2], 2)
    gmsh.model.geo.addCurveLoop([  8,  9, 10, -6], 3)
    gmsh.model.geo.addCurveLoop([ 11, 12, 13, -3], 4)
    gmsh.model.geo.addCurveLoop([ -7, 14, 15,-11], 5)
    gmsh.model.geo.addCurveLoop([-10, 16, 17,-14], 6)
    gmsh.model.geo.addCurveLoop([-12, 18, 19, 20], 7)
    gmsh.model.geo.addCurveLoop([-15, 21, 22,-18], 8)
    gmsh.model.geo.addCurveLoop([-17, 23, 24,-21], 9)
    gmsh.model.geo.addCurveLoop([ 25, 26], 10)
    #gmsh.model.geo.addCurveLoop([ 25, 26,31], 10)
    
    gmsh.model.geo.addPlaneSurface([ 1],  1)
    gmsh.model.geo.addPlaneSurface([ 2],  2)
    gmsh.model.geo.addPlaneSurface([ 3],  3)
    gmsh.model.geo.addPlaneSurface([ 4],  4)
    gmsh.model.geo.addPlaneSurface([ 5,10],  5)
    gmsh.model.geo.addPlaneSurface([ 6],  6)
    gmsh.model.geo.addPlaneSurface([ 7],  7)
    gmsh.model.geo.addPlaneSurface([ 8],  8)
    gmsh.model.geo.addPlaneSurface([ 9],  9)
    gmsh.model.geo.addPlaneSurface([10], 10)
    gmsh.model.geo.synchronize()
    gmsh.model.mesh.embed(1, [27], 2, 5)
    gmsh.model.mesh.embed(1, [28], 2, 5)
    #gmsh.model.mesh.embed(1, [29], 2, 5)
    #gmsh.model.mesh.embed(1, [30], 2, 5)

    # Physical groups
    gmsh.model.addPhysicalGroup(0, [1,2,3,4,5,8,9,12,13,14,15,16], 1)
    gmsh.model.setPhysicalName(0, 1, "DirichletNodes")
    gmsh.model.addPhysicalGroup(0, [17, 19], 2)
    gmsh.model.setPhysicalName(0, 2, "DesignNodes")
    gmsh.model.addPhysicalGroup(1, [1,4,5,8,9,13,16,20,19,22,24,23], 3)
    gmsh.model.setPhysicalName(1, 3, "DirichletEdges")
    gmsh.model.addPhysicalGroup(1, [25,26], 4)
    gmsh.model.setPhysicalName(1, 4, "DesignEdges")
    gmsh.model.addPhysicalGroup(2, [1,2,3,4,6,7,8,9], 5)
    gmsh.model.setPhysicalName(2, 5, "PML")
    gmsh.model.addPhysicalGroup(2, [10], 6)
    gmsh.model.setPhysicalName(2, 6, "Design")
    gmsh.model.addPhysicalGroup(2, [5], 7)
    gmsh.model.setPhysicalName(2, 7, "Air")
    gmsh.model.addPhysicalGroup(1, [27, 28], 8)
    gmsh.model.setPhysicalName(1, 8, "Target")
    gmsh.model.mesh.generate(2)
    
    # ... and save it to disk
    gmsh.write(meshfile_name)
    gmsh.finalize()
end

λ = 1.0           # Wavelength
L = 2.5           # Width of the rectangular domain
H = 2.5           # Height of the rectangular domain
rd = 0.5          # Radius of the design domain circle
rt = rd + 0.2     # Radius of the target circle
dpml = 0.5        # Thickness of PML
# Characteristic length (controls the resolution, smaller the finer)
resol = 25.0      # Number of points per wavelength
l1 = λ/resol      # Normal region
l2 = l1/2.0       # Design region
l3 = 2*l1         # PML

geo_param = CirRecGeometry(L, H, rd, rt, dpml, l1, l2, l3)
meshfile = "geometry.msh"
MeshGenerator(geo_param, meshfile)